In [ ]:
!pip install ydata-synthetic
!pip install pandas-profiling

## Importing necessary libraries

In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import sklearn.cluster as cluster
from numpy import array, random, sum, unique
from pandas import DataFrame, read_csv

from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
from ydata_synthetic.synthesizers.regular import WGAN_GP
from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

## Loading the data

In [2]:
df = pd.read_csv(r'data/diabetes_binary_health_indicators_BRFSS2015.csv')

## Running pandas-profiling to explore the data

In [3]:
profile = ProfileReport(df, title="Pandas Profiling Report for Diabetes Dataset", explorative=True)

In [5]:
profile.to_widgets()

### Saving the report as a HTML file

In [6]:
profile.to_file(r"pandas_profiling_reports/Diabetes_Pandas_Profiling_Report.html")

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Dropping duplicate rows

In [7]:
df.drop_duplicates(inplace = True)

## Creating train test splits

In [8]:
#Creating a test split from 20% of the data for testing the model
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(df.drop(['Diabetes_binary'], axis = 1), df['Diabetes_binary'], test_size = 0.2, stratify = df['Diabetes_binary'], random_state = 0)

In [9]:
#Adding the labels to the X_train_df
X_train_df['Diabetes_binary'] = y_train_df

## Data Synthesizing

### Creating synthetic data for minority class

In [6]:
#WGAN with Gradient Penalty is chosen as the GAN architecture
model = WGAN_GP

In [7]:
#Choosing the categorical and numerical columns
num_cols = ['BMI', 'MentHlth', 'PhysHlth']
cat_cols = list(X_train_df.drop(num_cols, axis = 1).columns)

In [8]:
#Selecting the rows of minority class from the data
train_data = X_train_df.loc[ X_train_df['Diabetes_binary']==1 ].copy()

In [9]:
print(f"Dataset info: Number of rows - {train_data.shape[0]}") 
print(f"Number of columns - {train_data.shape[1]}")

Dataset info: Number of rows - 28078
Number of columns - 22


In [10]:
#Setting the parameters of the GAN model
noise_dim = 32
dim = 128
batch_size = 128

log_step = 100
epochs = 200+1
learning_rate = 5e-4
beta_1 = 0.5
beta_2 = 0.9
models_dir = './cache'

In [11]:
#Setting the model parameters and the training step parameters of the GAN model
gan_args = ModelParameters(batch_size=batch_size, lr=learning_rate, betas=(beta_1, beta_2),
                           noise_dim=noise_dim,layers_dim=dim)

train_args = TrainParameters(epochs=epochs, sample_interval=log_step)

In [12]:
#Initializing the GAN model
synthesizer = model(gan_args, n_critic = 10)

2022-07-28 08:33:11.501809: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [13]:
#Training the GAN model
synthesizer.train(data = train_data, train_arguments = train_args, num_cols = num_cols, cat_cols = cat_cols)

2022-07-28 08:33:13.257143: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
  0%|▏                                          | 1/201 [00:17<58:41, 17.61s/it]

Epoch: 0 | disc_loss: -0.30025714635849 | gen_loss: 0.15275044739246368


  1%|▍                                          | 2/201 [00:26<42:20, 12.77s/it]

Epoch: 1 | disc_loss: -0.3248370885848999 | gen_loss: 0.22272323071956635


  1%|▋                                          | 3/201 [00:36<37:06, 11.24s/it]

...

Epoch: 198 | disc_loss: -0.34156543016433716 | gen_loss: 0.33427584171295166


100%|████████████████████████████████████████▊| 200/201 [32:19<00:10, 10.20s/it]

Epoch: 199 | disc_loss: -0.3508546054363251 | gen_loss: 0.323633074760437


100%|█████████████████████████████████████████| 201/201 [32:29<00:00,  9.70s/it]

Epoch: 200 | disc_loss: -0.3494935929775238 | gen_loss: 0.32680952548980713


#### Generating 100k rows of minority data

In [14]:
#Generating synthetic data of 100k examples of diabetic patients
minority_synth_data = synthesizer.sample(100000)

Synthetic data generation: 100%|██████████████| 782/782 [00:11<00:00, 67.44it/s]


In [15]:
minority_synth_data.isna().sum()

HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
Diabetes_binary         0
dtype: int64

In [16]:
minority_synth_data

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,1.0,0.0,0.0,32.280758,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,2.0,0.308237,20.622051,1.0,1.0,7.0,1.0,1.0,1.0
1,0.0,1.0,0.0,29.729445,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,5.0,0.447694,6.936053,1.0,0.0,5.0,3.0,4.0,1.0
2,1.0,0.0,0.0,33.646648,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,3.0,-0.253246,10.605074,1.0,0.0,7.0,2.0,2.0,1.0
3,0.0,0.0,0.0,30.537703,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.033439,5.332734,1.0,0.0,8.0,2.0,7.0,1.0
4,1.0,1.0,1.0,30.573626,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,2.0,2.031053,14.967583,0.0,1.0,2.0,4.0,7.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100091,0.0,1.0,0.0,32.422222,1.0,1.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.527633,11.749678,1.0,0.0,1.0,3.0,4.0,1.0
100092,1.0,0.0,1.0,32.185829,1.0,0.0,1.0,0.0,1.0,1.0,...,1.0,2.0,1.175031,10.448512,1.0,0.0,7.0,5.0,1.0,1.0
100093,1.0,1.0,1.0,34.277805,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,-2.275481,25.303354,1.0,0.0,9.0,5.0,5.0,1.0
100094,0.0,0.0,1.0,35.670673,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,4.0,2.090659,5.257822,1.0,1.0,5.0,2.0,8.0,1.0


### Generating 100k rows of majority data

In [27]:
#Selecting the rows of majority class from the data
train_data = X_train_df.loc[ X_train_df['Diabetes_binary']==0 ].copy()

#WGAN with Gradient Penalty is chosen as the GAN architecture
model = WGAN_GP

#Setting the parameters of the GAN model
noise_dim = 32
dim = 128
batch_size = 128

log_step = 100
epochs = 100+1
learning_rate = 5e-4
beta_1 = 0.5
beta_2 = 0.9
models_dir = './cache'

#Setting the model parameters and the training step parameters of the GAN model
gan_args = ModelParameters(batch_size=batch_size, lr=learning_rate, betas=(beta_1, beta_2),
                           noise_dim=noise_dim,layers_dim=dim)

train_args = TrainParameters(epochs=epochs, sample_interval=log_step)

#Initializing the GAN model
synthesizer = model(gan_args, n_critic = 20)

#Training the GAN model
synthesizer.train(data = train_data, train_arguments = train_args, num_cols = num_cols, cat_cols = cat_cols)

#Generating synthetic data of 100k examples of non-diabetic patients
majority_synth_data = synthesizer.sample(100000)

  1%|          | 1/101 [01:43<2:52:01, 103.22s/it]

Epoch: 0 | disc_loss: -0.3417395055294037 | gen_loss: 0.17075596749782562


  2%|▏         | 2/101 [03:00<2:24:52, 87.80s/it] 

Epoch: 1 | disc_loss: -0.35966137051582336 | gen_loss: 0.21103143692016602


  3%|▎         | 3/101 [04:16<2:14:41, 82.46s/it]

...

 99%|█████████▉| 100/101 [2:09:34<01:18, 78.30s/it]

Epoch: 99 | disc_loss: -0.3502044081687927 | gen_loss: 0.3703605532646179


100%|██████████| 101/101 [2:10:53<00:00, 77.75s/it]


Epoch: 100 | disc_loss: -0.3563690781593323 | gen_loss: 0.3735984265804291


Synthetic data generation: 100%|██████████| 782/782 [00:16<00:00, 48.75it/s]


In [29]:
majority_synth_data.isna().sum()

HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
Diabetes_binary         0
dtype: int64

In [31]:
majority_synth_data.shape

(100096, 22)

In [34]:
minority_synth_data.shape

(100096, 22)

In [32]:
synth_df = pd.concat([majority_synth_data, minority_synth_data], ignore_index=True)

In [35]:
synth_df.shape

(200192, 22)

In [33]:
synth_df.to_csv(r'./generated_data/diabetes_binary_health_synthetic_data.csv', index = False)

### Generating pandas-profiling report for synthetic data

In [10]:
synth_df = pd.read_csv(r'./generated_data/diabetes_binary_health_synthetic_data.csv')

In [11]:
profile = ProfileReport(synth_df, title="Pandas Profiling Report for Synthetic Diabetes Dataset", explorative=True)

In [12]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/anaconda3/envs/ydata/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4868: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
#Saving the profile report of synthetic data as HTML file
profile.to_file(r"pandas_profiling_reports/Synthetic_Diabetes_Pandas_Profiling_Report.html")

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Modelling

### Replacing the negative values in columns "MentHlth" and "PhysHlth" with zero

In [78]:
synth_df['MentHlth'][synth_df['MentHlth'] < 0] = 0
synth_df['PhysHlth'][synth_df['PhysHlth'] < 0] = 0

In [79]:
synth_df.MentHlth.describe()

count    200192.000000
mean          1.868842
std           4.080144
min           0.000000
25%           0.000000
50%           0.000000
75%           1.407046
max          29.953527
Name: MentHlth, dtype: float64

In [80]:
synth_df.PhysHlth.describe()

count    200192.000000
mean         15.033990
std          12.740442
min           0.000000
25%           2.229514
50%          19.804619
75%          27.963614
max          29.140305
Name: PhysHlth, dtype: float64

### Splitting the X and y from the synthetic data for modelling

In [81]:
X = synth_df.drop(['Diabetes_binary'], axis = 1)
y = synth_df['Diabetes_binary']

### Building a ML model (using XGBoost)

In [127]:
from lightgbm import LGBMClassifier
from sklearn.svm import SVC

In [136]:
model = LGBMClassifier(random_state = 0)

In [137]:
model.fit(X, y)

LGBMClassifier(random_state=0)

### Evaluating the model on the test data

In [138]:
y_pred = model.predict(X_test_df)

In [140]:
from sklearn.metrics import classification_report
print(classification_report(y_test_df, y_pred))

              precision    recall  f1-score   support

         0.0       0.86      0.89      0.88     38876
         1.0       0.27      0.23      0.25      7019

    accuracy                           0.79     45895
   macro avg       0.57      0.56      0.56     45895
weighted avg       0.77      0.79      0.78     45895

